In [21]:
import numpy as np
import sklearn.model_selection
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [22]:
data = pd.read_csv('train.csv')

In [23]:
train, val = sklearn.model_selection.train_test_split(data)

In [24]:
train_X, train_cat_y = train.drop('label', axis=1).values, train['label'].values
val_X, val_cat_y = val.drop('label', axis=1).values, val['label'].values

In [25]:
def one_hot(label, categories=10):
    ret_val = np.zeros(categories)
    ret_val[label] = 1
    return ret_val

train_y = np.zeros((len(train_X), 10))
val_y = np.zeros((len(val_X), 10))

for i in range(len(train_y)):
    train_y[i] = one_hot(train_cat_y[i])

for i in range(len(val_y)):
    val_y[i] = one_hot(val_cat_y[i])

In [52]:
def to_2D(one_d):
    return one_d.reshape(len(one_d),28,28)

train_X = to_2D(train_X)
val_X = to_2D(val_X)

In [59]:
model = tf.keras.Sequential()

model.add(layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(28,28)))
model.add(layers.Conv1D(32, kernel_size=3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [60]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [63]:
model.fit(train_X, train_y, epochs=30, batch_size=32, 
          validation_data=(val_X, val_y))

Train on 31500 samples, validate on 10500 samples
Epoch 1/10
31500/31500 [==============================] - 9s 271us/step - loss: 0.0358 - acc: 0.9905 - val_loss: 0.1265 - val_acc: 0.9757
Epoch 2/10
31500/31500 [==============================] - 10s 308us/step - loss: 0.0262 - acc: 0.9930 - val_loss: 0.1515 - val_acc: 0.9719
Epoch 3/10
31500/31500 [==============================] - 8s 269us/step - loss: 0.0360 - acc: 0.9910 - val_loss: 0.2089 - val_acc: 0.9627
Epoch 4/10
31500/31500 [==============================] - 8s 269us/step - loss: 0.0407 - acc: 0.9898 - val_loss: 0.1634 - val_acc: 0.9735
Epoch 5/10
31500/31500 [==============================] - 9s 274us/step - loss: 0.0283 - acc: 0.9925 - val_loss: 0.1587 - val_acc: 0.9755
Epoch 6/10
31500/31500 [==============================] - 9s 277us/step - loss: 0.0293 - acc: 0.9934 - val_loss: 0.1884 - val_acc: 0.9721
Epoch 7/10
31500/31500 [==============================] - 8s 268us/step - loss: 0.0336 - acc: 0.9915 - val_loss: 0.1656 -

In [64]:
# test data
test_data = pd.read_csv('test.csv')
test_X = to_2D(test_data.values)

In [65]:
y_pred = model.predict(test_X)

In [66]:
y_cat_pred = []

for i in range(len(y_pred)):
    y_cat_pred.append([int(i + 1), int(np.argmax(y_pred[i]))])

In [67]:
import csv

with open('predictions.csv', 'w') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(['ImageId', 'Label'])
    for row in y_cat_pred:
        writer.writerow(row)